## Results Section 6.2: Better Decompositions
We present the results for Section 6.2 here. Specifically, we reproduce table 4 here.

### Imports and setup

In [1]:
import pandas as pd
import os
from qiskit import QuantumCircuit, QuantumRegister
import subprocess
from analyzer import main_analysis

source_file = "../../data/times.csv"
source_output = '../../data/output'
output_folder = "../../data/paper"

### Defining helper functions

In [2]:
def add_rccx(circuit, c1, c2, t1):
    """
    Adds an RCCX gate to the given circuit.

    Parameters:
    circuit (QuantumCircuit): The quantum circuit to add the RCCX gate to.
    c1 (int): The control qubit 1.
    c2 (int): The control qubit 2.
    t1 (int): The target qubit.

    Returns:
    QuantumCircuit: The circuit with the RCCX gate added.
    """
    circuit.h(t1)
    circuit.cx(t1, c1)
    circuit.t(c1)
    circuit.cx(c2, t1)
    circuit.cx(c2, c1)
    circuit.t(c1)
    circuit.tdg(t1)
    circuit.cx(c2, t1)
    circuit.tdg(t1)
    circuit.cx(c2, c1)
    circuit.cx(c2, t1)
    circuit.cx(c1, t1)
    circuit.cx(c2, c1)
    circuit.cx(c2, t1)
    circuit.h(c1)
    circuit.cx(c1, t1)
    circuit.cx(t1, c1)
    return circuit

def add_ciswap(circuit, c1, c2, t1):
    """
    Adds a controlled iSWAP gate to the given circuit.

    Args:
        circuit (QuantumCircuit): The quantum circuit to add the gate to.
        c1 (int): The control qubit 1.
        c2 (int): The target qubit 1.
        t1 (int): The target qubit 2.

    Returns:
        None
    """
    circuit.cx(t1,c2)
    circuit.h(t1)
    circuit.cx(t1,c2)
    circuit.cx(c2,c1)
    circuit.t(c2)
    circuit.tdg(c1)
    circuit.cx(t1,c2)
    circuit.cx(c2,c1)
    circuit.tdg(t1)
    circuit.cx(t1,c2)
    circuit.t(c1)
    circuit.cx(c2,c1)
    circuit.cx(t1,c2)
    circuit.h(t1)
    circuit.cx(c2,c1)
    circuit.cx(t1,c2)
    circuit.cx(t1,c2)

def controlled_circuit(circuit, implementations):
    """
    Constructs a controlled version of the given quantum circuit using the provided implementations.

    Args:
        circuit (QuantumCircuit): The input quantum circuit.
        implementations (dict): A dictionary containing the controlled implementations of the gates.

    Returns:
        QuantumCircuit: The controlled version of the input circuit.
    """

    num_qubits = circuit.num_qubits

    new_qr = QuantumRegister(num_qubits + 1 + 1, name="q")

    controlled_circ = QuantumCircuit(new_qr, name="controlled_circuit")

    for inst, qargs, _ in circuit.data:
        gate = inst
        target_qubits = [new_qr[q.index + 1] for q in qargs]

        if gate.name in implementations:
            if gate.name == 't' or gate.name == "tdg":
                implementation_qubits = [new_qr[0]] + target_qubits + [new_qr[-1]]
            else:
                implementation_qubits = [new_qr[0]] + target_qubits

            implementation = implementations[gate.name]
            controlled_circ.compose(implementation, qubits=implementation_qubits, inplace=True)

        else:
            controlled_gate = gate.control(1)
            controlled_circ.append(controlled_gate, [new_qr[0]] + target_qubits)

    return controlled_circ

def save_circuit_to_qasm(circuit, filename):
    """
    Save a quantum circuit to a QASM file.
    
    Parameters:
        circuit (QuantumCircuit): The quantum circuit to be saved.
        filename (str): The name of the QASM file to save the circuit to.
    """
    qasm_str = circuit.qasm()
    with open(filename, 'w') as file:
        file.write(qasm_str)

## Loading and returning the best circuits

In [3]:
outputs = []
os.makedirs(os.path.join(output_folder, '62'), exist_ok=True)
for folder in os.listdir(os.path.join(source_output, '62')):
    output = dict()
    output['operator'] = folder
    # main analysis analyzes the output and returns the best circuit
    _, _, _, _, best_t_depth_circ, _ = main_analysis(os.path.join(source_output, '62', folder))
    output['t_depth'] = best_t_depth_circ.t_depth
    output['qubits'] = best_t_depth_circ.circuit.num_qubits

    controlled_implementations = {
        "cx": QuantumCircuit.from_qasm_file("../../data/baselines/controlled/ccx.qasm"),
        "t": QuantumCircuit.from_qasm_file("../../data/baselines/controlled/ct.qasm"),
        "h": QuantumCircuit.from_qasm_file("../../data/baselines/controlled/ch.qasm"),
        "s": QuantumCircuit.from_qasm_file("../../data/baselines/controlled/cs.qasm"),
        "sdg": QuantumCircuit.from_qasm_file("../../data/baselines/controlled/cs_inverse.qasm"),
        "tdg": QuantumCircuit.from_qasm_file("../../data/baselines/controlled/ct_inverse.qasm"),
    }

    # adding the baselines
    if folder == 'rcccx':
        output['baseline_t_depth'] = 8 # from paper https://arxiv.org/abs/1508.03273
    elif folder == 'cct':
        circuit = QuantumCircuit(5)
        add_rccx(circuit, 0, 1, 3)
        # add controlled t gate
        add_rccx(circuit, 2,3,4)
        circuit.t(4)
        add_rccx(circuit, 2,3,4)
        add_rccx(circuit, 0, 1, 3)
        output['baseline_t_depth'] = circuit.depth(lambda gate: gate[0].name in ['t', 'tdg'])
    elif folder == 'cciswap':
        circuit = QuantumCircuit(5) # using https://threeplusone.com/pubs/on_gates.pdf
        add_rccx(circuit, 0, 1, 4)
        add_ciswap(circuit, 2,3,4)
        add_rccx(circuit, 0, 1, 4)
        output['baseline_t_depth'] = circuit.depth(lambda gate: gate[0].name in ['t', 'tdg'])
    elif folder == 'csqrtswap':
        circuit = QuantumCircuit(2)
        circuit.cx(1,0)
        circuit.h(1)
        circuit.t(1)
        circuit.cx(0,1)
        circuit.tdg(1)
        circuit.tdg(0)
        circuit.h(1)
        circuit.cx(1,0)
        circuit.sdg(1)
        circuit.s(0)
        controlled_circ = controlled_circuit(circuit, controlled_implementations)
        save_circuit_to_qasm(controlled_circ, f"../../data/input/baseline.qasm")
        # running simplification for honest comparison
        output_subprocess = subprocess.check_output(f"cd .. && cd .. && ./bin/main_resynth baseline.qasm", shell=True)
        output_ = output_subprocess.decode("utf-8").split("T-depth: ")[1].split(" -> ")[1]
        # remove the baseline file
        os.remove(f"../../data/input/baseline.qasm")
        int_tdepth = int(output_)
        output['baseline_t_depth'] = int_tdepth
    else:
        circuit = QuantumCircuit(2)
        circuit.cx(1,0)
        circuit.h(1)
        circuit.tdg(1)
        circuit.cx(0,1)
        circuit.tdg(1)
        circuit.h(1)
        circuit.sdg(1)
        circuit.s(0)
        circuit.h(1)
        circuit.cx(1,0)
        circuit.sdg(0)
        controlled_circ = controlled_circuit(circuit, controlled_implementations)
        save_circuit_to_qasm(controlled_circ, f"../../data/input/baseline.qasm")
        # running simplification for honest comparison
        output_subprocess = subprocess.check_output(f"cd .. && cd .. && ./bin/main_resynth baseline.qasm", shell=True)
        output_ = output_subprocess.decode("utf-8").split("T-depth: ")[1].split(" -> ")[1]
        os.remove(f"../../data/input/baseline.qasm")
        int_tdepth = int(output_)
        output['baseline_t_depth'] = int_tdepth

    best_t_depth_circ.circuit.qasm(formatted=True, filename=os.path.join(output_folder, '62', f'{folder}.qasm'))
    outputs.append(output)

outputs = pd.DataFrame(outputs)
outputs.to_csv(os.path.join(output_folder, '62', 'table4.csv'), index=False)
outputs

100%|██████████| 931/931 [01:18<00:00, 11.90it/s]


OPENQASM 2.0;
include "qelib1.inc";
qreg qubits[4];
h qubits[3];
cx qubits[3],qubits[1];
t qubits[1];
tdg qubits[3];
cx qubits[3],qubits[1];
h qubits[3];
cx qubits[3],qubits[0];
t qubits[0];
cx qubits[2],qubits[3];
tdg qubits[3];
cx qubits[2],qubits[3];
cx qubits[2],qubits[0];
t qubits[0];
tdg qubits[3];
cx qubits[0],qubits[3];
cx qubits[2],qubits[0];
h qubits[0];
cx qubits[2],qubits[3];
cx qubits[0],qubits[3];
cx qubits[3],qubits[0];
cx qubits[1],qubits[0];
sdg qubits[0];
t qubits[3];
cx qubits[1],qubits[3];
cx qubits[1],qubits[0];
tdg qubits[3];
h qubits[3];
cx qubits[0],qubits[3];



100%|██████████| 540/540 [00:39<00:00, 13.68it/s]


OPENQASM 2.0;
include "qelib1.inc";
qreg qubits[4];
h qubits[3];
cx qubits[3],qubits[2];
cx qubits[0],qubits[3];
t qubits[0];
t qubits[2];
tdg qubits[3];
cx qubits[0],qubits[3];
cx qubits[0],qubits[2];
t qubits[2];
tdg qubits[3];
cx qubits[2],qubits[3];
cx qubits[0],qubits[2];
cx qubits[0],qubits[3];
h qubits[0];
cx qubits[0],qubits[1];
t qubits[1];
h qubits[2];
cx qubits[2],qubits[3];
cx qubits[3],qubits[0];
tdg qubits[0];
cx qubits[3],qubits[0];
tdg qubits[0];
cx qubits[3],qubits[1];
t qubits[1];
cx qubits[1],qubits[0];
cx qubits[3],qubits[1];
h qubits[1];
cx qubits[3],qubits[0];
cx qubits[1],qubits[0];
cx qubits[0],qubits[1];
tdg qubits[0];
h qubits[0];
cx qubits[0],qubits[1];
t qubits[1];
cx qubits[3],qubits[0];
tdg qubits[0];
cx qubits[3],qubits[0];
tdg qubits[0];
cx qubits[3],qubits[1];
t qubits[1];
cx qubits[1],qubits[0];
cx qubits[3],qubits[2];
cx qubits[3],qubits[1];
h qubits[1];
cx qubits[3],qubits[0];
cx qubits[1],qubits[0];
h qubits[3];
cx qubits[3],qubits[2];
cx qubits[0],

100%|██████████| 15/15 [00:01<00:00, 10.10it/s]


...

100%|██████████| 3237/3237 [04:29<00:00, 12.00it/s]
/tmp/ipykernel_3967881/1038651528.py:84: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  target_qubits = [new_qr[q.index + 1] for q in qargs]


OPENQASM 2.0;
include "qelib1.inc";
qreg qubits[3];
cx qubits[1],qubits[0];
h qubits[1];
tdg qubits[1];
h qubits[1];
cx qubits[1],qubits[2];
cx qubits[0],qubits[1];
tdg qubits[1];
cx qubits[0],qubits[1];
tdg qubits[1];
t qubits[2];
cx qubits[0],qubits[2];
t qubits[2];
cx qubits[2],qubits[1];
cx qubits[0],qubits[2];
h qubits[2];
cx qubits[2],qubits[1];
cx qubits[0],qubits[1];
cx qubits[1],qubits[2];
t qubits[1];
h qubits[1];
cx qubits[1],qubits[0];
tdg qubits[0];
t qubits[1];
cx qubits[2],qubits[1];
tdg qubits[1];
cx qubits[2],qubits[0];
t qubits[0];
s qubits[2];



100%|██████████| 371/371 [00:29<00:00, 12.60it/s]


OPENQASM 2.0;
include "qelib1.inc";
qreg qubits[4];
cx qubits[0],qubits[3];
cx qubits[3],qubits[0];
cx qubits[1],qubits[3];
h qubits[1];
cx qubits[1],qubits[0];
t qubits[0];
s qubits[1];
h qubits[1];
cx qubits[1],qubits[3];
cx qubits[2],qubits[1];
tdg qubits[1];
cx qubits[2],qubits[1];
tdg qubits[1];
t qubits[3];
cx qubits[2],qubits[3];
t qubits[3];
cx qubits[3],qubits[1];
cx qubits[2],qubits[3];
h qubits[3];
cx qubits[3],qubits[1];
cx qubits[2],qubits[1];
cx qubits[1],qubits[3];
cx qubits[1],qubits[0];
t qubits[1];
s qubits[1];
cx qubits[0],qubits[1];
tdg qubits[0];
sdg qubits[0];
h qubits[0];
cx qubits[0],qubits[2];
s qubits[1];
h qubits[1];
t qubits[2];
cx qubits[3],qubits[0];
tdg qubits[0];
cx qubits[3],qubits[0];
tdg qubits[0];
cx qubits[3],qubits[2];
t qubits[2];
cx qubits[2],qubits[0];
cx qubits[3],qubits[2];
h qubits[2];
cx qubits[3],qubits[0];
cx qubits[2],qubits[0];
cx qubits[0],qubits[2];
s qubits[0];
s qubits[2];
cx qubits[3],qubits[0];
sdg qubits[3];
sdg qubits[3];
cx qubi

,operator,t_depth,qubits,baseline_t_depth
0,rcccx,5,4,8
1,cct,9,4,9
2,cciswap,8,4,6
3,csqrtiswap,6,3,27
4,csqrtswap,6,4,29
